## これまでの重みをローカルからDBに移行するためのプロセス

In [1]:
import os
import base64

directory = 'models'
file_dict = {}

for filename in os.listdir(directory):
    if filename.endswith('.h5'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'rb') as file:
            encoded_data = base64.b64encode(file.read()).decode('utf-8')
            key = os.path.splitext(filename)[0]
            file_dict[key] = encoded_data


In [4]:
len(file_dict)

146

In [6]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
load_dotenv()
uri = os.getenv('MongoDBURI')
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
# client.admin.command('ping')
db = client.get_database('ml')
assert db is not None, "db is None"
collection = db.get_collection('h5')

In [9]:
already_done = set([item['_id'] for item in list(collection.find({}, {'_id': 1}))])

In [ ]:
file_dict = {k: v for k, v in file_dict.items() if k not in already_done}

In [ ]:
collection.insert_many([{'_id': key, 'model': value} for key, value in file_dict.items()])